In [81]:
from utils_misc import pickle_load_or_create
import pandas as pd
import numpy as np
import cv2


In [82]:
history = pickle_load_or_create('../models/last_cnn_model_history', lambda: {})

In [83]:
pd.DataFrame(history.history)

,accuracy,loss,val_accuracy,val_loss
0,0.468750,1.322263,0.506944,1.270124
1,0.487847,1.267614,0.513889,1.265558
2,0.482639,1.236608,0.513889,1.267687
3,0.482639,1.190585,0.493056,1.251327
4,0.489583,1.157687,0.541667,1.191321
5,0.505208,1.124633,0.541667,1.147607
6,0.534722,1.053638,0.569444,1.086957
7,0.598958,0.983012,0.638889,1.020759
8,0.614583,0.923050,0.680556,0.922151
9,0.618056,0.863846,0.631944,0.893664


In [84]:
from DataGeneratorFrames import DataGeneratorSkillBorders

In [85]:
def get_random_frame(videonames, videofolder):
    """
    videos: array of video_paths
    df_video_border_labels: panda dataframe ['path', 'frame', 'borderlabel']
        [ 0 : ground
          1 : heels of ground
          2 : air ]
    """
    vid_id = np.random.randint(0, len(videonames)-1)
    vid_name = videonames[vid_id]
    path = videofolder + vid_name
    print(path)
    cap = cv2.VideoCapture(path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_nr = np.random.randint(0, video_length-1)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    res, frame = cap.read()
    cap.release()
    # cv2.destroyAllWindows()

    return vid_name, frame_nr, frame

In [86]:
path, frame_nr, frame = get_random_frame(['20240209_atelier_009.mp4', '20240209_atelier_009.mp4'], '../videos/')

../videos/20240209_atelier_009.mp4


In [87]:
path, frame_nr

('20240209_atelier_009.mp4', 39)

In [88]:
def show_frame(frame, scale=0.4):
    """
    Displays a single frame using OpenCV, waits for a key press, 
    and then closes the frame window.

    Args:
        frame (numpy.ndarray): The image frame to display.

    Returns:
        int: The ASCII value of the key pressed.
    """
    if scale != 1.0:
        frame = cv2.resize(frame, dsize=(0,0), fx=scale, fy=scale)
    # Display the frame using OpenCV
    cv2.imshow('Frame', frame)
    
    # Wait for a key press
    key = cv2.waitKey(0)
    cv2.destroyAllWindows()
    return key

In [89]:
show_frame(frame, scale=0.4)

113

In [90]:
def load_frame(path, frame_nr, dx, dy):
    cap = cv2.VideoCapture(path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    res, frame = cap.read()
    if not res:
        raise ValueError(f"Failed to read frame {frame_nr} from {path}")
    frame = cv2.resize(frame, (dx, dy))
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Uncomment if necessary
    cap.release()
    return frame / 255.0

In [91]:
frame64 = load_frame('../videos/20240209_atelier_009.mp4', frame_nr, 64, 64) 

In [92]:
frame64.shape

(64, 64, 3)

In [93]:
tensor = np.expand_dims(frame64, axis=0)
tensor.shape

(1, 64, 64, 3)

In [94]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# Example prediction
logits = history.model.predict(tensor)

# Convert to probabilities
probabilities = softmax(logits)
np.round(probabilities, 2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


array([[0.04, 0.  , 0.03, 0.  , 0.93]], dtype=float32)